In [1]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import random
import time
import datetime
import io
from konlpy.tag import *

In [2]:
dlg_list=[]
with io.open("./Fashion-How/data/ac_eval_t1.dev", encoding='euc-kr') as f:
    for a in f:
        b = a.strip().split('\t')
        dlg_list.append(b)

In [3]:
type(dlg_list[10][1].strip().split())
dlg_list[0][0]

'; 0'

In [4]:
con_list = []
rnk1_list = []
rnk2_list = []
rnk3_list = []
con_dlg_temp = []
for dlg in dlg_list:
    if ';' in dlg[0] and dlg[0].strip() != '; 0':
        con_list.append(con_dlg_temp)
        con_dlg_temp = []
    if dlg[0] == 'CO' or dlg[0] == 'US':
        con_dlg_temp.append(dlg[1])
    elif dlg[0] == 'R1':
        rnk1_list.append(dlg[1].strip().split())
    elif dlg[0] == 'R2':
        rnk2_list.append(dlg[1].strip().split())
    elif dlg[0] == 'R3':
        rnk3_list.append(dlg[1].strip().split())

In [5]:
con_list[199]

['안녕하세요. 코디봇입니다. 무엇을 도와드릴까요?',
 '자격증 시험을 보러 가려고 하는데 적당한 두께의 트레이닝 복을 추천해주세요.',
 '아우터도 추천해드릴까요?',
 '아니요.',
 '신발은 운동화로 추천해드릴까요?',
 '네.']

In [6]:
seq_con_list = [" ".join(s) for s in con_list]
for idx, sen in enumerate(seq_con_list):
    seq_con_list[idx] = "".join(s for s in sen if s != "." and s != ",")

In [7]:
def dicmmdata(filepath):
    with io.open(filepath, encoding='euc-kr') as md:
        pre_cloth = ""
        cloth_dic = {}
        for a in md:
            key = a.strip().split("\t")[0].strip()
            value = a.strip().split("\t")[4].strip()
            value = "".join(letter for letter in value if letter !=  "." or letter !=  "," or letter !=  "!" or letter !=  "?"or letter !=  "/")
            if key != pre_cloth:
                cloth_dic[key]=[value]
            else:
                cloth_dic[key].append(value)
            pre_cloth = key.strip()
    return cloth_dic

In [8]:
print(len(seq_con_list))
print(len(rnk1_list))

200
200


In [9]:
md_rnk1_list = []
md_rnk2_list = []
md_rnk3_list = []
cloth_dic = dicmmdata("Fashion-How/data/mdata.txt")

for idx, li in enumerate(rnk1_list):
    for jdx, cl in enumerate(li):
        rnk1_list[idx][jdx] = seq_con_list[idx] + " ".join(cloth_dic[cl.strip()])
for idx, li in enumerate(rnk2_list):
    for jdx, cl in enumerate(li):
        rnk2_list[idx][jdx] = seq_con_list[idx] + " ".join(cloth_dic[cl.strip()])
for idx, li in enumerate(rnk3_list):
    for jdx, cl in enumerate(li):
        rnk3_list[idx][jdx] = seq_con_list[idx] + " ".join(cloth_dic[cl.strip()])

In [10]:
rnk1_list[:20]

[['가을 축제에 입고갈 스타일로 코디해주세요 치마 바지 원피스 중 어떤 옷이 포함된 코디를 추천해드릴까요? 원피스나 치마로 추천해주세요 가을에 입기 좋은 적당한 두께감의 아우터와 함께 추천해드릴까요? 네 상의는 무늬없는 티로 추천해드리는 것은 어떠신가요? 좋습니다블루종 점퍼 셔츠 칼라 단추 앞여밈 둥근 스냅 단추 5개 부드러운 어깨 라인 드롭 숄더 여유로운 소매 폭 손목 위까지 오는 소매 길이 소매단 고무밴딩 처리 밑단 고무 밴딩 처리 엉덩이를 가리는 전체 길이 체크 패턴 안감 아방한 핏 좌우 아래에 싱글 웰트 포켓 면100% 탄탄한 소재감 신축성 없음 드라이크리닝 권장 아이보리 컬러 심플한 컬러 다른 컬러들과 매치 가능 라이트톤 데일리룩 깔끔한 느낌 베이직한 디자인 캐주얼한 간절기용 아우터로 입기 좋은 아이템',
  '가을 축제에 입고갈 스타일로 코디해주세요 치마 바지 원피스 중 어떤 옷이 포함된 코디를 추천해드릴까요? 원피스나 치마로 추천해주세요 가을에 입기 좋은 적당한 두께감의 아우터와 함께 추천해드릴까요? 네 상의는 무늬없는 티로 추천해드리는 것은 어떠신가요? 좋습니다얕은 라운드 네크라인 슬림한 일자 소매통 일자형 암홀 손등까지 내려오는 소매기장 허리선까지 오는 기장 전체적으로 슬림한 라인 네크라인 재원단 처리 소매끝과 밑단 삼봉 봉제 처리 앞, 뒤 길이 차이 없는 일자형 밑단 적당히 몸에 붙는 슬림한 스타일 심플하고 베이직한 아이템 면 100% 싱글 저지 부드럽고 편안한 느낌의 신축성 좋음 두께감 얇음 베이지 다크 그레이쉬 톤의 내추럴하고 온화한 느낌 편안한 캐주얼한 슬림한 심플한',
  '가을 축제에 입고갈 스타일로 코디해주세요 치마 바지 원피스 중 어떤 옷이 포함된 코디를 추천해드릴까요? 원피스나 치마로 추천해주세요 가을에 입기 좋은 적당한 두께감의 아우터와 함께 추천해드릴까요? 네 상의는 무늬없는 티로 추천해드리는 것은 어떠신가요? 좋습니다허벅지 윗쪽 길이의 미니 스커트 A라인 실루엣 밴드허리, 벨트고리 있음, 허리 뒤쪽 고무밴딩 벨트 탈착 가

In [11]:
mecab = Mecab('C:\mecab\mecab-ko-dic')


In [12]:
temp = []
for idx, le in enumerate(rnk1_list):
    for jdx ,_ in enumerate(le):
        temp.append(rnk1_list[idx][jdx])
        temp.append(rnk2_list[idx][jdx])
        temp.append(rnk3_list[idx][jdx])
temp[:10]

['가을 축제에 입고갈 스타일로 코디해주세요 치마 바지 원피스 중 어떤 옷이 포함된 코디를 추천해드릴까요? 원피스나 치마로 추천해주세요 가을에 입기 좋은 적당한 두께감의 아우터와 함께 추천해드릴까요? 네 상의는 무늬없는 티로 추천해드리는 것은 어떠신가요? 좋습니다블루종 점퍼 셔츠 칼라 단추 앞여밈 둥근 스냅 단추 5개 부드러운 어깨 라인 드롭 숄더 여유로운 소매 폭 손목 위까지 오는 소매 길이 소매단 고무밴딩 처리 밑단 고무 밴딩 처리 엉덩이를 가리는 전체 길이 체크 패턴 안감 아방한 핏 좌우 아래에 싱글 웰트 포켓 면100% 탄탄한 소재감 신축성 없음 드라이크리닝 권장 아이보리 컬러 심플한 컬러 다른 컬러들과 매치 가능 라이트톤 데일리룩 깔끔한 느낌 베이직한 디자인 캐주얼한 간절기용 아우터로 입기 좋은 아이템',
 '가을 축제에 입고갈 스타일로 코디해주세요 치마 바지 원피스 중 어떤 옷이 포함된 코디를 추천해드릴까요? 원피스나 치마로 추천해주세요 가을에 입기 좋은 적당한 두께감의 아우터와 함께 추천해드릴까요? 네 상의는 무늬없는 티로 추천해드리는 것은 어떠신가요? 좋습니다블루종 점퍼 셔츠 칼라 단추 앞여밈 둥근 스냅 단추 5개 부드러운 어깨 라인 드롭 숄더 여유로운 소매 폭 손목 위까지 오는 소매 길이 소매단 고무밴딩 처리 밑단 고무 밴딩 처리 엉덩이를 가리는 전체 길이 체크 패턴 안감 아방한 핏 좌우 아래에 싱글 웰트 포켓 면100% 탄탄한 소재감 신축성 없음 드라이크리닝 권장 아이보리 컬러 심플한 컬러 다른 컬러들과 매치 가능 라이트톤 데일리룩 깔끔한 느낌 베이직한 디자인 캐주얼한 간절기용 아우터로 입기 좋은 아이템',
 '가을 축제에 입고갈 스타일로 코디해주세요 치마 바지 원피스 중 어떤 옷이 포함된 코디를 추천해드릴까요? 원피스나 치마로 추천해주세요 가을에 입기 좋은 적당한 두께감의 아우터와 함께 추천해드릴까요? 네 상의는 무늬없는 티로 추천해드리는 것은 어떠신가요? 좋습니다블루종 점퍼 셔츠 칼라 단추 앞여밈 둥근 스냅 단추 5개 부드러운 어깨 라인 

In [13]:
max = 0
for tl in temp:
    length = len(tl.strip().split())
    if max < length:
        max = length
print(max)

179


In [14]:
for idx, le in enumerate(rnk1_list):
    for jdx ,_ in enumerate(le):
        word1 = mecab.morphs(rnk1_list[idx][jdx])
        word2 = mecab.morphs(rnk2_list[idx][jdx])
        word3 = mecab.morphs(rnk3_list[idx][jdx])
        rnk1_list[idx][jdx] = " ".join(word1)
        rnk2_list[idx][jdx] = " ".join(word2)
        rnk3_list[idx][jdx] = " ".join(word3)

In [15]:
rnk1_list[:20]

[['가을 축제 에 입 고 갈 스타일 로 코디 해 주 세요 치마 바지 원피스 중 어떤 옷 이 포함 된 코디 를 추천 해드릴까요 ? 원피스 나 치마 로 추천 해 주 세요 가을 에 입 기 좋 은 적당 한 두께 감 의 아우 터 와 함께 추천 해드릴까요 ? 네 상의 는 무늬 없 는 티 로 추천 해 드리 는 것 은 어떠신가요 ? 좋 습니다 블루종 점퍼 셔츠 칼라 단추 앞여밈 둥근 스냅 단추 5 개 부드러운 어깨 라인 드롭 숄더 여유 로운 소매 폭 손목 위 까지 오 는 소매 길 이 소 매단 고무 밴 딩 처리 밑단 고무 밴 딩 처리 엉덩이 를 가리 는 전체 길 이 체크 패턴 안감 아방 한 핏 좌우 아래 에 싱글 웰트 포켓 면 100 % 탄탄 한 소재 감 신축성 없음 드라이 크리 닝 권장 아이보리 컬러 심플 한 컬러 다른 컬러 들 과 매치 가능 라이트 톤 데일리 룩 깔끔 한 느낌 베이직 한 디자인 캐주얼 한 간 절기 용 아우 터로 입기 좋 은 아이템',
  '가을 축제 에 입 고 갈 스타일 로 코디 해 주 세요 치마 바지 원피스 중 어떤 옷 이 포함 된 코디 를 추천 해드릴까요 ? 원피스 나 치마 로 추천 해 주 세요 가을 에 입 기 좋 은 적당 한 두께 감 의 아우 터 와 함께 추천 해드릴까요 ? 네 상의 는 무늬 없 는 티 로 추천 해 드리 는 것 은 어떠신가요 ? 좋 습니다 얕 은 라운드 네크라인 슬림 한 일자 소매통 일자 형 암홀 손등 까지 내려오 는 소매 기장 허리선 까지 오 는 기장 전체 적 으로 슬림 한 라인 네크라인 재 원단 처리 소매 끝 과 밑단 삼봉 봉제 처리 앞 , 뒤 길 이 차이 없 는 일자 형 밑단 적당히 몸 에 붙 는 슬림 한 스타일 심플 하고 베이직 한 아이템 면 100 % 싱글 저 지 부드럽 고 편안 한 느낌 의 신축성 좋 음 두께 감 얇 음 베이지 다크 그레 이쉬 톤 의 내추럴 하 고 온화 한 느낌 편안 한 캐주얼 한 슬림 한 심플 한',
  '가을 축제 에 입 고 갈 스타일 로 코디 해 주 세요 치마 바지 원피스 중 어떤 옷 이 포함 된 코디

In [16]:
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 512

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    text_len = len(input_ids)
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [17]:
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)

    # 그래디언트 계산 안함
    with torch.no_grad():
        # Forward 수행
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [28]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
device = torch.device("cuda")
model = torch.load("data/model/bert-4.pt")
model.eval()
predict_rank= []
#
for idx, le in enumerate(rnk1_list):
        pred1 = 0.0
        pred2 = 0.0
        pred3 = 0.0
        for jdx in range(len(le)):
            temp1 = test_sentences([rnk1_list[idx][jdx]])
            temp2 = test_sentences([rnk2_list[idx][jdx]])
            temp3 = test_sentences([rnk3_list[idx][jdx]])
            pred1 = pred1 +  temp1[0][1] - temp1[0][0]
            pred2 = pred2 +  temp2[0][1] - temp2[0][0]
            pred3 = pred3 +  temp3[0][1] - temp3[0][0]
        predict_rank.append([pred1,pred2,pred3])
print(predict_rank)

[[-0.9961145026609302, -0.9955957885831594, -0.9957611570134759], [-0.7463007925543934, -0.7330095304641873, -0.7429334095213562], [-0.7477911964524537, -0.7328093193937093, -0.7477233002427965], [-0.746160525130108, -0.7409592086914927, -0.74735859897919], [-0.4972668304108083, -0.49769065948203206, 0.23396474146284163], [-0.9959154995158315, -0.9959048759192228, -0.9953335579484701], [-0.9971895115450025, -0.9968384066596627, -0.996734463609755], [-1.1337179413530976, -1.1404639680404216, -1.1405225249473006], [-0.7478769936133176, -0.7471176462713629, -0.7469389068428427], [-0.7471166329924017, -0.7474757984746248, -0.7473722996655852], [-0.9959390806034207, -0.9949444038793445, -0.33283292897976935], [-0.7477779549080878, -0.7434259748551995, -0.7342027130071074], [-0.9960361309349537, -0.9915994899347425, -0.9956352906301618], [-0.9905518079176545, -0.9905328918248415, -0.9903703052550554], [0.3121394596528262, -0.9875403670594096, -0.9916978571563959], [-0.9776484000030905, -0.97

In [36]:
predict_rank

[[-0.9961145026609302, -0.9955957885831594, -0.9957611570134759],
 [-0.7463007925543934, -0.7330095304641873, -0.7429334095213562],
 [-0.7477911964524537, -0.7328093193937093, -0.7477233002427965],
 [-0.746160525130108, -0.7409592086914927, -0.74735859897919],
 [-0.4972668304108083, -0.49769065948203206, 0.23396474146284163],
 [-0.9959154995158315, -0.9959048759192228, -0.9953335579484701],
 [-0.9971895115450025, -0.9968384066596627, -0.996734463609755],
 [-1.1337179413530976, -1.1404639680404216, -1.1405225249473006],
 [-0.7478769936133176, -0.7471176462713629, -0.7469389068428427],
 [-0.7471166329924017, -0.7474757984746248, -0.7473722996655852],
 [-0.9959390806034207, -0.9949444038793445, -0.33283292897976935],
 [-0.7477779549080878, -0.7434259748551995, -0.7342027130071074],
 [-0.9960361309349537, -0.9915994899347425, -0.9956352906301618],
 [-0.9905518079176545, -0.9905328918248415, -0.9903703052550554],
 [0.3121394596528262, -0.9875403670594096, -0.9916978571563959],
 [-0.97764840

In [35]:
test_sentences(["안녕 하 세요 코디 봇 입니다 무엇 을 도와드릴까요 ? 영화 보 러 가 는데 가디건 이 포함 된 룩 으로 추천 해 주 세요 색상 이 비슷 한 아이템 들 로 구성 된 코디 를 추천 해드릴까요 ? 좋 아요 히든 굽 블 로 퍼 슬리퍼 뮬 스티치 라인 속 굽 3 CM 겉 굽 1 CM 합성 피혁 스웨이드 터치 브라운 카키 브라운 따뜻 한 편안 한 차분 한 투박 한 부드러운"])
test_sentences(["안녕 하 세요 코디 봇 입니다 무엇 을 도와드릴까요 ? 편하 면서 고급 스러운 룩 으로 추천 해 주 세요 치마 바지 원피스 중 어떤 옷 이 포함 된 코디 를 추천 해드릴까요 ? 바지 로 추천 해 주 세요 아 우터 는 코트 와 자켓 중 어떤 아이템 을 추천 해드릴까요 ? 자켓 으로 추천 해 주 세요 세미 베이직 핏 9 부 슬 렉스 앞 중심 지퍼 여 밈 처리 훅 앤 아이 처리 , 버튼 여 밈 이중 처리 밴드 허리 처리 밑위 가 김 세미 부츠 컷 형태 무릎 부터 약간 퍼지 는 세미 와이드 앞 중심 레그 라인 심플 한 핀턱 처리 앞 슬랜트 주머니 뒤 더블 입술 포켓 , 포켓 위 다트 처리 혼용 율 : 리넨 100 % 얇 은 두께 로 비침 있 음 리넨 소재 로 통풍 성 이 좋 음 신축성 없 음 단독 세탁 권장 세탁 후 다림질 권장 라이트 그레 이쉬 톤 ( 저명 도 저채 도의 ) 베이지 컬러 차분 하 고 고급 스러운 이미지 시원 하 고 활동 적 인 편안 해 보이 는 다리 가 길 어 보이 는 고급 스럽 고 깔끔 한 이미지 캐주얼 룩 오피스 룩 차분 한 단정 한 세련 된 모던 한"])

array([[ 0.00188822, -0.24761611]], dtype=float32)

In [45]:
temp_df = pd.DataFrame
order_list = []
for idx, dwk in enumerate(predict_rank):
    temp_list=[]
    del temp_df
    for jdx in range(3):
        temp_list.append([predict_rank[idx][jdx],jdx])
    print(temp_list)
    temp_df = pd.DataFrame(temp_list,columns=["value","idx"])
    order_list.append(list(temp_df.sort_values(by=['value'],axis=0)["idx"]))

[[-0.9961145026609302, 0], [-0.9955957885831594, 1], [-0.9957611570134759, 2]]
[[-0.7463007925543934, 0], [-0.7330095304641873, 1], [-0.7429334095213562, 2]]
[[-0.7477911964524537, 0], [-0.7328093193937093, 1], [-0.7477233002427965, 2]]
[[-0.746160525130108, 0], [-0.7409592086914927, 1], [-0.74735859897919, 2]]
[[-0.4972668304108083, 0], [-0.49769065948203206, 1], [0.23396474146284163, 2]]
[[-0.9959154995158315, 0], [-0.9959048759192228, 1], [-0.9953335579484701, 2]]
[[-0.9971895115450025, 0], [-0.9968384066596627, 1], [-0.996734463609755, 2]]
[[-1.1337179413530976, 0], [-1.1404639680404216, 1], [-1.1405225249473006, 2]]
[[-0.7478769936133176, 0], [-0.7471176462713629, 1], [-0.7469389068428427, 2]]
[[-0.7471166329924017, 0], [-0.7474757984746248, 1], [-0.7473722996655852, 2]]
[[-0.9959390806034207, 0], [-0.9949444038793445, 1], [-0.33283292897976935, 2]]
[[-0.7477779549080878, 0], [-0.7434259748551995, 1], [-0.7342027130071074, 2]]
[[-0.9960361309349537, 0], [-0.9915994899347425, 1], [

In [44]:
temp_df.sort_values(by=['value'],axis=0)
list(temp_df.sort_values(by=['value'],axis=0)["idx"])

[2, 0, 1]

In [48]:
def changekau(order):
    if order == [2,1,0]:
        return 5
    elif order == [2,0,1]:
        return 4
    elif order == [1,2,0]:
        return 3
    elif order == [1,0,2]:
        return 2
    elif order == [0,2,1]:
        return 1
    elif order == [0,1,2]:
        return 0


In [52]:
kau_list=[]
for aklw in order_list:
    kau_list.append(changekau(aklw))
kau_list = np.array(kau_list)
np.savetxt("./prediction.csv", kau_list.astype(int), encoding='utf8', fmt='%d')